## Import Libraries 

In [14]:
import pandas as pd

## Read Data

In [19]:
raw_train_data = pd.read_csv('dataset/vs_train.csv')
raw_validation_data = pd.read_csv('dataset/vs_dev.csv')

raw_train_data

,Unnamed: 0,tweet,label
0,0,Sip N Shop Come thru right now #Marjais #Popul...,0
1,1,I don't know about you but My family and I wil...,1
2,2,@MSignorile Immunizations should be mandatory....,2
3,3,President Obama spoke in favor of vaccination ...,0
4,4,"""@myfoxla: Arizona monitoring hundreds for mea...",0
...,...,...,...
15971,15971,@Salon if u believe the anti-vax nutcases caus...,1
15972,15972,How do you feel about parents who don't #vacci...,0
15973,15973,70 Preschoolers Tested for Measles in Simi Val...,0
15974,15974,Finance Minister: Budget offers room to procur...,0
